In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
import itertools
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.optimizers import Adam

import sklearn.model_selection as sk
from sklearn.preprocessing import MinMaxScaler

In [30]:
df = pd.read_csv('health_data.csv')

# loads the data that will be used as input
X = df[['Age','Sex','HighChol','CholCheck','BMI','Smoker','HeartDiseaseorAttack','PhysActivity','Fruits','Veggies','HvyAlcoholConsump','GenHlth','MentHlth','PhysHlth','DiffWalk']]
# loads the data that will be used as output
yDiabetes = df[['Diabetes']]

In [31]:
X_train, X_test, y_train, y_test = sk.train_test_split(X, yDiabetes, test_size=0.2, random_state = 42)

In [32]:
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.fit_transform(X_test)

In [33]:
NEW_y_train = []
for i in y_train.to_numpy():
    NEW_y_train.append(int(i[0]))

https://towardsdatascience.com/how-to-find-optimal-neural-network-architecture-with-tensorflow-the-easy-way-50575a03d060

In [34]:
# total number of layers
num_layers = 3
# minimmum number of nodes
min_nodes_per_layer = 64
# maximum number of nodes
max_nodes_per_layer = 256
# amount to increase/decrease the number of nodes each step
node_step_size = 64

node_options = list(range(
    min_nodes_per_layer, 
    max_nodes_per_layer + 1, 
    node_step_size
))

In [35]:
# gives us all the possible combinations/permutaions of the neural network
layer_possibilities = [node_options] * num_layers
layer_node_permutations = list(itertools.product(*layer_possibilities))

In [36]:
models = []

for permutation in layer_node_permutations:
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(15,)))
    model_name = ''
    
    for nodes_at_layer in permutation:
        model.add(tf.keras.layers.Dense(nodes_at_layer, activation='relu'))
        model_name += f'dense{nodes_at_layer}_'
        
    model.add(tf.keras.layers.Dense(2, activation='softmax'))
    model._name = model_name[:-1]
    
    models.append(model)

In [ ]:
trainingSummary = []

for model in models:
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(x=scaled_X_train, y=np.array(NEW_y_train), batch_size=10, epochs=30, shuffle=True, verbose=2)
    
    preds = model.predict(scaled_X_test)
    
    
    prediction_classes = pd.DataFrame(np.argmax(preds, axis=-1))

    ans = {
        'model_name': model.name,
        'model_number': len(trainingSummary),
        'test_accuracy': accuracy_score(y_test, prediction_classes),
        'test_precision': precision_score(y_test, prediction_classes),
        'test_recall': recall_score(y_test, prediction_classes),
        'test_f1': f1_score(y_test, prediction_classes)
    }
    
    trainingSummary.append(ans)
    
    

In [ ]:
pd.DataFrame(trainingSummary).sort_values(by='test_accuracy', ascending=False)

,model_name,test_accuracy,test_precision,test_recall,test_f1
20,20,0.748921,0.730167,0.787346,0.757679
44,44,0.748073,0.728117,0.789474,0.757555
34,34,0.747365,0.731461,0.779401,0.754670
42,42,0.747295,0.728986,0.784934,0.755926
21,21,0.747153,0.720041,0.806356,0.760758
...,...,...,...,...,...
29,29,0.741707,0.706304,0.824940,0.761026
40,40,0.741283,0.738299,0.745212,0.741740
28,28,0.739656,0.693519,0.856150,0.766301
15,15,0.739303,0.703843,0.823663,0.759053
